In [12]:
%load_ext autoreload
%autoreload 2

from humbldata.toolbox.toolbox_controller import Toolbox

toolbox = Toolbox(
    symbol=2,
    interval="1d",
    start_date="2021-01-01",
    end_date="2021-01-01"
)

# toolbox.technical.mandelbrot_channel(range="1m")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ValidationError: 1 validation error for Toolbox
symbol
  Value error, `symbol` must be a `str` [type=value_error, input_value=2, input_type=int]
    For further information visit https://errors.pydantic.dev/2.6/v/value_error

In [4]:
toolbox

Toolbox(symbol=AAPL,AMD, interval=1d, start_date=2021-01-01, end_date=2021-01-01, provider=yfinance)